In [ ]:
import docker
client = docker.from_env()
import socket
code = """\
#include "nanobench.h"
#include <iostream>
#include <fstream>

struct Point { int x, y; };

void func() {
    const int N = 1000;
    std::vector<std::vector<int>> matrix(N, std::vector<int>(N, 1));
    long long sum = 0;
    // Column-major traversal
    for (int j = 0; j < N; ++j) {
        for (int i = 0; i < N; ++i) {
            sum += matrix[i][j];
        }
    }
    std::cout << sum << std::endl;
}

int main() {
    std::ofstream nullStream("/dev/null");
    auto* oldBuf = std::cout.rdbuf(nullStream.rdbuf());
    ankerl::nanobench::Bench benchmark;
    benchmark.output(nullptr);
    benchmark.run("test", []() {func();});
    std::cout.rdbuf(oldBuf);
    benchmark.render(ankerl::nanobench::templates::pyperf(), std::cout);
}
"""

compileCommand = (
    "g++ "
    "-O3 --std=c++20 "
    "nanobench.o "
    "-x c++ - "
    "-o a.out"
)

shellCommand = (
    "sh -c \""
    f"{compileCommand}; "
    "exitCode=$?; "
    "if [ $exitCode -eq 0 ]; then ./a.out; "
    "else echo Compilation failed; fi"
    "\""
)

container = client.containers.run(
    "localhost/benchmark-examples", 
    shellCommand,
    working_dir="/usr/src",
    remove=True, detach=True, stdin_open=True,
)
socketIO = container.attach_socket(params=dict(stdin=1, stdout=1, stderr=1, stream=1))
sock = socketIO._sock
sock.sendall(code.encode())
sock.shutdown(socket.SHUT_WR)
while chunk:=sock.recv(1024):
    print(chunk.decode(errors="ignore"), end="")

     {
    "benchmarks": [
        {
            "runs": [
                {
                    "values": [
                        0.00274842,
                        0.002104871,
                        0.002041735,
                        0.002033847,
                        0.002040551,
                        0.002023144,
                        0.002012453,
                        0.002031788,
                        0.002043128,
                        0.002022587,
                        0.002038246
                    ]
                }
            ]
        }
    ],
    "metadata": {
        "loops": 11,
        "inner_loops": 1,
        "name": "benchmark",
        "unit": "second"
    },
    "version": "1.0"
}

In [ ]:
res = container.exec_run(
    f"sh -c \"{compileCommand} && ./a.out\"",
    workdir="/usr/src",
    stdin=True, stdout=True, stderr=True, socket=True
)

In [4]:
res.output

In [5]:
sock = res.output._sock
sock.sendall(code.encode())
sock.shutdown(socket.SHUT_WR)
while chunk:=sock.recv(1024):
    print(chunk.decode(errors="ignore"), end="")

      Warning, results might be unstable:
* CPU frequency scaling enabled: CPU 0 between 1,000.0 and 3,900.0 MHz
* Turbo is enabled, CPU frequency will fluctuate

Recommendations
* Use 'pyperf system tune' before benchmarking. See https://github.com/psf/pyperf
      
|               ns/op |                op/s |    err% |     total | benchmark
|--------------------:|--------------------:|--------:|----------:|:----------
|        2,081,826.00 |              480.35 |    0.3% |      0.02 | `test`


In [19]:
"{{{txt}}}".format(txt="foo")

'{foo}'

In [ ]:
"""\
#include "nanobench.h"
{includes}

struct Point { int x, y; };

void func() {{
    {body}
}}

int main() {
    std::ofstream nullStream("/dev/null");
    auto* oldBuf = std::cout.rdbuf(nullStream.rdbuf());
    ankerl::nanobench::Bench benchmark;
    benchmark.output(nullptr);
    benchmark.run("test", []() {func();});
    std::cout.rdbuf(oldBuf);
    benchmark.render(ankerl::nanobench::templates::pyperf(), std::cout);
}
""" % {"includes" : "#include <iostream>", "body": "int i;"}

ValueError: unsupported format character '{' (0x7b) at index 24